In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import mstats
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

C:\Users\Gebruiker\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_excel('Final dataset for real3.xlsx', index_col = 0)

In [4]:
data['Country ISO code'] = data['Country ISO code'].astype('category')
data['Size'] = data['Size'].astype('category')
data['Accounting practice'] = data['Accounting practice'].astype('category')
data['NACE Rev. 2'] = data['NACE Rev. 2'].astype('category')

X = data[["Country ISO code","Nr. of Tax Treaties","NACE Rev. 2","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities", "Size","Accounting practice","Degree","Closeness","Betweenness","Eigenvector","Hubs"]]
y = data["ETR"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state= 2031996, shuffle = True)

X_train_con = X_train[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_train_cat = X_train[['NACE Rev. 2', 'Size', 'Accounting practice']]

scaler = StandardScaler()
scaler.fit(X_train_con)
StandardScaler()
X_train_con = scaler.transform(X_train_con)

iim=IterativeImputer(#estimator=xgb.XGBRegressor(),
initial_strategy='median',
max_iter=10,
missing_values= np.nan,
random_state=2031996)

imputed_X_train_con = pd.DataFrame(iim.fit_transform(X_train_con), index = X_train.index)
imputed_X_train_con.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

imputed_X_train_cat = pd.get_dummies(X_train_cat)
imputed_X_train = pd.concat([imputed_X_train_con, imputed_X_train_cat], axis = 1)

drop_na = pd.concat([imputed_X_train, y_train], axis = 1)
drop_na = drop_na.dropna()
drop_na = pd.DataFrame(drop_na)

X_train_final = drop_na.loc[:,:'Accounting practice_US GAAP']
y_train_final = drop_na['ETR']

X_test_con = X_test[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_test_cat = X_test[['NACE Rev. 2', 'Size', 'Accounting practice']]

X_test_con = scaler.transform(X_test_con)

imputed_X_test_con = pd.DataFrame(iim.transform(X_test_con), index = X_test.index)
imputed_X_test_con.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

imputed_X_test_cat = pd.get_dummies(X_test_cat)
imputed_X_test = pd.concat([imputed_X_test_con, imputed_X_test_cat], axis = 1)

drop_na = pd.concat([imputed_X_test, y_test], axis = 1)
#drop_na = drop_na.dropna()
drop_na = pd.DataFrame(drop_na)

X_test_final = drop_na.loc[:,:'Accounting practice_US GAAP']
y_test_final = drop_na['ETR']

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)
print('Imputed' , X_train_final.shape, y_train_final.shape)

Train (16986, 20) (16986,)
Test (4247, 20) (4247,)
Imputed (16986, 29) (16986,)


In [5]:
def winsorize(dataframe, upper, lower, column_to_winsor):
    dataframe = dataframe.sort_values(column_to_winsor, ascending=False, ignore_index = True)
    amount = dataframe[column_to_winsor].count()
    up = int(amount * (upper/100))
    low = int(amount * (lower/100))
    data_winsor = dataframe.loc[low:up]
    return data_winsor

data_winsor = winsorize(data, 99,  1, 'ETR') 
data_winsor = winsorize(data_winsor, 99, 1, "Operating revenue")
data_winsor = winsorize(data_winsor, 99, 1, "Total assets")
data_winsor = winsorize(data_winsor, 99, 1, "Current liabilities")
data_winsor = winsorize(data_winsor, 99, 1, "Betweenness")
data_winsor = winsorize(data_winsor, 99, 1, "Long term debt")
data_winsor = winsorize(data_winsor, 99, 1, "PBT")

X_winsor = data_winsor[["Nr. of Tax Treaties","NACE Rev. 2","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities", "Size","Accounting practice","Degree","Closeness","Betweenness","Eigenvector","Hubs"]]
y_winsor = data_winsor["ETR"]

X_train_winsor, X_test_winsor, y_train_winsor, y_test_winsor = train_test_split(X_winsor, y_winsor, test_size=0.20, random_state= 2031996, shuffle = True)

X_train_con_winsor = X_train_winsor[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_train_cat_winsor = X_train_winsor[['NACE Rev. 2', 'Size', 'Accounting practice']]

scaler = StandardScaler()
scaler.fit(X_train_con_winsor)
StandardScaler()
X_train_con_winsor = scaler.transform(X_train_con_winsor)

iim=IterativeImputer(#estimator=xgb.XGBRegressor(),
initial_strategy='median',
max_iter=10,
missing_values= np.nan,
#skip_complete=True,
random_state=2031996)

imputed_X_train_con_winsor = pd.DataFrame(iim.fit_transform(X_train_con_winsor), index = X_train_winsor.index)
imputed_X_train_con_winsor.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

#imputed_X_train = imputed_X_train.dropna()
imputed_X_train_cat_winsor = pd.get_dummies(X_train_cat_winsor)
imputed_X_train_winsor = pd.concat([imputed_X_train_con_winsor, imputed_X_train_cat_winsor], axis = 1)

drop_na_winsor = pd.concat([imputed_X_train_winsor, y_train_winsor], axis = 1)
drop_na_winsor = drop_na_winsor.dropna()
drop_na_winsor = pd.DataFrame(drop_na_winsor)

X_train_final_winsor = drop_na_winsor.loc[:,:'Accounting practice_US GAAP']
y_train_final_winsor = drop_na_winsor['ETR']

X_test_con_winsor = X_test_winsor[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_test_cat_winsor = X_test_winsor[['NACE Rev. 2', 'Size', 'Accounting practice']]

X_test_con_winsor = scaler.transform(X_test_con_winsor)

imputed_X_test_con_winsor = pd.DataFrame(iim.transform(X_test_con_winsor), index = X_test_winsor.index)
imputed_X_test_con_winsor.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

imputed_X_test_cat_winsor = pd.get_dummies(X_test_cat_winsor)
imputed_X_test_winsor = pd.concat([imputed_X_test_con_winsor, imputed_X_test_cat_winsor], axis = 1)

drop_na_winsor = pd.concat([imputed_X_test_winsor, y_test_winsor], axis = 1)
#drop_na = drop_na.dropna()
drop_na_winsor = pd.DataFrame(drop_na_winsor)

X_test_final_winsor = drop_na_winsor.loc[:,:'Accounting practice_US GAAP']
y_test_final_winsor = drop_na_winsor['ETR']

print('Train', X_train_winsor.shape, y_train_winsor.shape)
print('Test', X_test_winsor.shape, y_test_winsor.shape)
print('Imputed train' , X_train_final_winsor.shape, y_train_final_winsor.shape)
print('Imputed test' , X_test_final_winsor.shape, y_test_final_winsor.shape)

Train (11108, 19) (11108,)
Test (2778, 19) (2778,)
Imputed train (11108, 29) (11108,)
Imputed test (2778, 29) (2778,)


In [6]:
def winsorize(dataframe, upper, lower, column_to_winsor):
    dataframe = dataframe.sort_values(column_to_winsor, ascending=False, ignore_index = True)
    amount = dataframe[column_to_winsor].count()
    up = int(amount * (upper/100))
    low = int(amount * (lower/100))
    data_winsor = dataframe.loc[low:up]
    return data_winsor

data_winsor2 = winsorize(data, 97.5,  4, 'ETR') 
data_winsor2 = winsorize(data_winsor2, 100,  4, "Operating revenue")
data_winsor2 = winsorize(data_winsor2, 100,  4, "Total assets")
data_winsor2 = winsorize(data_winsor2, 100,  4, "Current liabilities")
data_winsor2 = winsorize(data_winsor2, 100,  4, "Betweenness")
data_winsor2 = winsorize(data_winsor2, 100,  4, "Long term debt")
data_winsor2 = winsorize(data_winsor2, 97.5,  2.5, "PBT")

X_winsor2 = data_winsor2[["Nr. of Tax Treaties","NACE Rev. 2","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities", "Size","Accounting practice","Degree","Closeness","Betweenness","Eigenvector","Hubs"]]
y_winsor2 = data_winsor2["ETR"]

X_train_winsor2, X_test_winsor2, y_train_winsor2, y_test_winsor2 = train_test_split(X_winsor2, y_winsor2, test_size=0.20, random_state= 2031996, shuffle = True)

X_train_con_winsor2 = X_train_winsor2[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_train_cat_winsor2 = X_train_winsor2[['NACE Rev. 2', 'Size', 'Accounting practice']]

scaler = StandardScaler()
scaler.fit(X_train_con_winsor2)
StandardScaler()
X_train_con_winsor2 = scaler.transform(X_train_con_winsor2)

iim=IterativeImputer(#estimator=xgb.XGBRegressor(),
initial_strategy='median',
max_iter=10,
missing_values= np.nan,
#skip_complete=True,
random_state=2031996)

imputed_X_train_con_winsor2 = pd.DataFrame(iim.fit_transform(X_train_con_winsor2), index = X_train_winsor2.index)
imputed_X_train_con_winsor2.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

#imputed_X_train = imputed_X_train.dropna()
imputed_X_train_cat_winsor2 = pd.get_dummies(X_train_cat_winsor2)
imputed_X_train_winsor2 = pd.concat([imputed_X_train_con_winsor2, imputed_X_train_cat_winsor2], axis = 1)

drop_na_winsor2 = pd.concat([imputed_X_train_winsor2, y_train_winsor2], axis = 1)
drop_na_winsor2 = drop_na_winsor2.dropna()
drop_na_winsor2 = pd.DataFrame(drop_na_winsor2)

X_train_final_winsor2 = drop_na_winsor2.loc[:,:'Accounting practice_US GAAP']
y_train_final_winsor2 = drop_na_winsor2['ETR']

X_test_con_winsor2 = X_test_winsor2[["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]]
X_test_cat_winsor2 = X_test_winsor2[['NACE Rev. 2', 'Size', 'Accounting practice']]

X_test_con_winsor2 = scaler.transform(X_test_con_winsor2)

imputed_X_test_con_winsor2 = pd.DataFrame(iim.transform(X_test_con_winsor2), index = X_test_winsor2.index)
imputed_X_test_con_winsor2.columns = ["Nr. of Tax Treaties","Operating revenue","Number of employees",
          "PBT","Total assets","ROA","No of subsidiaries","Solvency ratio","Long term debt","Debtors",
          "Current liabilities"]

imputed_X_test_cat_winsor2 = pd.get_dummies(X_test_cat_winsor2)
imputed_X_test_winsor2 = pd.concat([imputed_X_test_con_winsor2, imputed_X_test_cat_winsor2], axis = 1)

drop_na_winsor2 = pd.concat([imputed_X_test_winsor2, y_test_winsor2], axis = 1)
#drop_na = drop_na.dropna()
drop_na_winsor2 = pd.DataFrame(drop_na_winsor2)

X_test_final_winsor2 = drop_na_winsor2.loc[:,:'Accounting practice_US GAAP']
y_test_final_winsor2 = drop_na_winsor2['ETR']

print('Train', X_train_winsor2.shape, y_train_winsor2.shape)
print('Test', X_test_winsor2.shape, y_test_winsor2.shape)
print('Imputed train' , X_train_final_winsor2.shape, y_train_final_winsor2.shape)
print('Imputed test' , X_test_final_winsor2.shape, y_test_final_winsor2.shape)

Train (8877, 19) (8877,)
Test (2220, 19) (2220,)
Imputed train (8877, 29) (8877,)
Imputed test (2220, 29) (2220,)


In [7]:
reg = LinearRegression().fit(X_train_final, y_train_final)
reg.score(X_test_final, y_test_final)
y_pred_final = reg.predict(X_test_final)
mae = mean_absolute_error(y_test_final, y_pred_final)
mse = mean_squared_error(y_test_final, y_pred_final)
rmse = np.sqrt(mse)
d_nonan = {'With outliers': [mse, rmse, mae]}
df_nonan = pd.DataFrame(d_nonan, index = ['MSE','RMSE','MAE'])
print(df_nonan)

      With outliers
MSE   150476.569392
RMSE     387.913095
MAE       95.938859


In [8]:
reg = LinearRegression().fit(X_train_final_winsor, y_train_final_winsor)
reg.score(X_test_final_winsor, y_test_final_winsor)
y_pred_final_winsor = reg.predict(X_test_final_winsor)
mae_winsor = mean_absolute_error(y_test_final_winsor, y_pred_final_winsor)
mse_winsor = mean_squared_error(y_test_final_winsor, y_pred_final_winsor)
rmse_winsor = np.sqrt(mse_winsor)
d_winsor = {'1% outliers removed': [mse_winsor, rmse_winsor, mae_winsor]}
df_winsor = pd.DataFrame(d_winsor, index = ['MSE','RMSE','MAE'])
print(df_winsor)

      1% outliers removed
MSE            833.649178
RMSE            28.872984
MAE             16.789811


In [9]:
reg = LinearRegression().fit(X_train_final_winsor2, y_train_final_winsor2)
reg.score(X_test_final_winsor2, y_test_final_winsor2)
y_pred_final_winsor2 = reg.predict(X_test_final_winsor2)
mae_winsor2 = mean_absolute_error(y_test_final_winsor2, y_pred_final_winsor2)
mse_winsor2 = mean_squared_error(y_test_final_winsor2, y_pred_final_winsor2)
rmse_winsor2 = np.sqrt(mse_winsor2)
d_winsor2 = {'5% outliers removed': [mse_winsor2, rmse_winsor2, mae_winsor2]}
df_winsor2 = pd.DataFrame(d_winsor2, index = ['MSE','RMSE','MAE'])
print(df_winsor2)

      5% outliers removed
MSE            260.408349
RMSE            16.137173
MAE             12.141384


In [10]:
results_LR = pd.concat([df_nonan, df_winsor, df_winsor2], axis = 1)
print(results_LR)

      With outliers  1% outliers removed  5% outliers removed
MSE   150476.569392           833.649178           260.408349
RMSE     387.913095            28.872984            16.137173
MAE       95.938859            16.789811            12.141384


In [11]:
results_LR.to_excel("results_LR_with_features.xlsx")